# Person Detection

In [1]:
experiment_name = 'person_detection'

import os
from detectron2.utils.logger import setup_logger
setup_logger()

from helpers.init import initialization

cfg, classes, labels_dict, \
train_dataset_combined, test_dataset_combined, valid_dataset_combined = initialization(merge_to_one_class = True, encoding = True)

### Training of the neural network ###

cfg.MODEL.WEIGHTS = "include/maskdino_swinl_50ep_300q_hid2048_3sd1_instance_maskenhanced_mask52.3ap_box59.0ap.pth" 

cfg.SOLVER.BACKBONE_MULTIPLIER = 1e-5

cfg.SOLVER.IMS_PER_BATCH = 2 # batch size
cfg.DATALOADER.NUM_WORKERS = 2
cfg.DATALOADER.PREFETCH_FACTOR = 2


cfg.SOLVER.BASE_LR = 1e-4
cfg.SOLVER.MAX_ITER = 5000 
cfg.SOLVER.STEPS = ([4000]) # when to decay learning rate

cfg.SOLVER.CHECKPOINT_PERIOD = 500
cfg.TEST.EVAL_PERIOD = 500

dir_list = os.listdir('output')
if '.ipynb_checkpoints' in dir_list:
        dir_list.remove('.ipynb_checkpoints')

cfg.OUTPUT_DIR = os.path.join('output', f'{len(dir_list)}')

 100% |█████████████████| 233/233 [1.0s elapsed, 0s remaining, 231.4 samples/s]         
 100% |███████████████████| 51/51 [280.8ms elapsed, 0s remaining, 182.9 samples/s]      
 100% |███████████████████| 50/50 [253.4ms elapsed, 0s remaining, 199.0 samples/s] 
Found train dataset labels:
['sitting', 'walking', 'waving']
Found test dataset labels:
['sitting', 'walking', 'waving']
Found valid dataset labels:
['sitting', 'walking', 'waving']
 100% |███████████████████| 74/74 [271.0ms elapsed, 0s remaining, 275.2 samples/s]      
 100% |███████████████████| 17/17 [90.3ms elapsed, 0s remaining, 188.3 samples/s]     
 100% |███████████████████| 17/17 [98.8ms elapsed, 0s remaining, 172.1 samples/s]     
Found difficult_train dataset labels:
['sitting', 'walking', 'waving']
Found difficult_test dataset labels:
['sitting', 'walking', 'waving']
Found difficult_valid dataset labels:
['sitting', 'walking', 'waving']
Label dictionary for complete dataset
{'person': 0}


## Training model

In [ ]:
import logging
import json

# setup and launch the trainer
from helpers.model_trainer import Trainer
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

os.makedirs(cfg.OUTPUT_DIR)

# dump labels for deployment
with open(os.path.join(cfg.OUTPUT_DIR, "classes.json"), "w") as fp:
    json.dump(classes, fp)

TRAIN = True
if TRAIN:
    trainer = Trainer(cfg, freeze=True, MaskDINO_weights=True)
    trainer.train()

## Loading model

In [3]:
from helpers.benchmark import benchmark

# if we just want to load the results of a previous model use that models experiment_name here
# else leave model_name at None to use the most recently trained model

model_name = None
#model_name = 'model_0000499.pth'
#cfg.OUTPUT_DIR = os.path.join('output', '4')

weights_path = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") if model_name is None else os.path.join(cfg.OUTPUT_DIR, model_name)
best_weights_path = os.path.join('include', '1_class_encoding_0.97.pth')

cfg.MODEL.WEIGHTS =  weights_path
#cfg.MODEL.WEIGHTS =  best_weights_path

# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1
# cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.8

benchmark(cfg, classes, labels_dict, train_dataset_combined, test_dataset_combined, valid_dataset_combined, save = True)

criterion.weight_dict  {'loss_ce': 4.0, 'loss_mask': 5.0, 'loss_dice': 5.0, 'loss_bbox': 5.0, 'loss_giou': 2.0, 'loss_ce_interm': 4.0, 'loss_mask_interm': 5.0, 'loss_dice_interm': 5.0, 'loss_bbox_interm': 5.0, 'loss_giou_interm': 2.0, 'loss_ce_dn': 4.0, 'loss_mask_dn': 5.0, 'loss_dice_dn': 5.0, 'loss_bbox_dn': 5.0, 'loss_giou_dn': 2.0, 'loss_ce_interm_dn': 4.0, 'loss_mask_interm_dn': 5.0, 'loss_dice_interm_dn': 5.0, 'loss_bbox_interm_dn': 5.0, 'loss_giou_interm_dn': 2.0, 'loss_ce_0': 4.0, 'loss_mask_0': 5.0, 'loss_dice_0': 5.0, 'loss_bbox_0': 5.0, 'loss_giou_0': 2.0, 'loss_ce_interm_0': 4.0, 'loss_mask_interm_0': 5.0, 'loss_dice_interm_0': 5.0, 'loss_bbox_interm_0': 5.0, 'loss_giou_interm_0': 2.0, 'loss_ce_dn_0': 4.0, 'loss_mask_dn_0': 5.0, 'loss_dice_dn_0': 5.0, 'loss_bbox_dn_0': 5.0, 'loss_giou_dn_0': 2.0, 'loss_ce_interm_dn_0': 4.0, 'loss_mask_interm_dn_0': 5.0, 'loss_dice_interm_dn_0': 5.0, 'loss_bbox_interm_dn_0': 5.0, 'loss_giou_interm_dn_0': 2.0, 'loss_ce_1': 4.0, 'loss_mask_1':

/home/user/seliuninas0/anaconda3/envs/maskdino/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


 100% |███████████████████| 68/68 [1.0s elapsed, 0s remaining, 66.7 samples/s]         
Performing IoU sweep...
 100% |███████████████████| 68/68 [984.4ms elapsed, 0s remaining, 69.1 samples/s]      
              precision    recall  f1-score   support

      person       0.98      0.97      0.97        96

   micro avg       0.98      0.97      0.97        96
   macro avg       0.98      0.97      0.97        96
weighted avg       0.98      0.97      0.97        96

VALIDATION DATASET

Directory 'output/test/inference' already exists; export will be merged with existing files
 100% |███████████████████| 67/67 [407.3ms elapsed, 0s remaining, 164.5 samples/s]      
Evaluating detections...
  45% |████████|----------| 30/67 [445.8ms elapsed, 549.9ms remaining, 67.3 samples/s] 

/home/user/seliuninas0/anaconda3/envs/maskdino/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


 100% |███████████████████| 67/67 [1.0s elapsed, 0s remaining, 62.5 samples/s]         
Performing IoU sweep...
 100% |███████████████████| 67/67 [902.5ms elapsed, 0s remaining, 74.2 samples/s]      
              precision    recall  f1-score   support

      person       0.95      0.93      0.94       100

   micro avg       0.95      0.93      0.94       100
   macro avg       0.95      0.93      0.94       100
weighted avg       0.95      0.93      0.94       100

TRAIN DATASET

Directory 'output/test/inference' already exists; export will be merged with existing files
 100% |█████████████████| 307/307 [2.0s elapsed, 0s remaining, 155.9 samples/s]      
Evaluating detections...
  47% |███████----------| 143/307 [1.9s elapsed, 2.3s remaining, 72.2 samples/s]    

/home/user/seliuninas0/anaconda3/envs/maskdino/lib/python3.8/site-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


 100% |█████████████████| 307/307 [4.6s elapsed, 0s remaining, 82.8 samples/s]      
Performing IoU sweep...
 100% |█████████████████| 307/307 [4.6s elapsed, 0s remaining, 77.9 samples/s]      
              precision    recall  f1-score   support

      person       0.97      0.98      0.97       437

   micro avg       0.97      0.98      0.97       437
   macro avg       0.97      0.98      0.97       437
weighted avg       0.97      0.98      0.97       437



In [4]:
# launch a fiftyone session to view the model predictions after training
import fiftyone as fo
session = fo.launch_app(train_dataset_combined, port = 6006)
session.wait()

Notebook sessions cannot wait


In [5]:
fo.close_app()